<a href="https://colab.research.google.com/github/sohaildua/PysparkLearning/blob/main/PysparkSql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive --quiet 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 199 kB 51.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c1c5ba9f46904ae9575253e7113d4fa3df42c717fdc2e716b11e120efe832f15
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSql").getOrCreate()
spark

# New Section

In [4]:
crime = spark.read.csv("rec-crime-pfa.csv",inferSchema=True, header=True)

In [5]:
crime.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling year total number of offences: integer (nullable = true)



In [6]:
crime.limit(4).show()

+----------------+-----------------+----------+--------------------+-------------------------------------+
|12 months ending|              PFA|    Region|             Offence|Rolling year total number of offences|
+----------------+-----------------+----------+--------------------+-------------------------------------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|                                25959|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft|                                 3090|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|                                26202|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|                                    2|
+----------------+-----------------+----------+--------------------+-------------------------------------+



In [7]:
crime.toPandas()

,12 months ending,PFA,Region,Offence,Rolling year total number of offences
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561
...,...,...,...,...,...
46464,31/12/2018,Wiltshire,South West,Stalking and harassment,2380
46465,31/12/2018,Wiltshire,South West,Theft from the person,347
46466,31/12/2018,Wiltshire,South West,Vehicle offences,2895
46467,31/12/2018,Wiltshire,South West,Violence with injury,5701


In [8]:
df = crime.withColumnRenamed('Rolling year total number of offences', 'CountOffences')

In [9]:
df

DataFrame[12 months ending: string, PFA: string, Region: string, Offence: string, CountOffences: int]

In [10]:
df.createOrReplaceTempView("TempView")

In [21]:
spark.sql("Select Region, Offence from TempView where CountOffences >300000").toPandas()

,Region,Offence
0,Fraud: CIFAS,CIFAS
1,Fraud: CIFAS,CIFAS
2,Fraud: CIFAS,CIFAS
3,Fraud: CIFAS,CIFAS
4,Fraud: Action Fraud,Action Fraud


In [27]:
spark.sql("select Region ,PFA , sum(CountOffences) from TempView GROUP BY Region,PFA").show()

+--------------------+-------------------+------------------+
|              Region|                PFA|sum(CountOffences)|
+--------------------+-------------------+------------------+
|                East|              Essex|           5938998|
|        Fraud: CIFAS|              CIFAS|           7678981|
|              London|Metropolitan Police|          42358281|
|          South East|               Kent|           6452567|
|       West Midlands|      West Midlands|          11320518|
|Yorkshire and The...|         Humberside|           4325685|
|                East|     Cambridgeshire|           3006281|
|                East|       Bedfordshire|           2328534|
|               Wales|        North Wales|           2307437|
|               Wales|        South Wales|           5309011|
|          South East|             Sussex|           5434316|
|          North West|            Cumbria|           1510798|
|          South West| Devon and Cornwall|           4991544|
|       